In [2]:
# Imports e arquivos
import pandas as pd
import matplotlib.pyplot as plt

path = r"C:\Users\Samsung\Desktop\Trabalho Despesas\DESPESAS_CEAPS-2008-2022.csv"
file = pd.read_csv(path, sep=';',low_memory=False)

In [3]:
# Transformando a coluna "VALOR_REEMBOLSADO" de string para float
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].astype(str)
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].str.replace(',','.')
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].str.replace('\r\n','')
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].astype(float)
    
# Removendo colunas desnecessárias para a atividade
df = file.drop(['CNPJ_CPF','FORNECEDOR','DOCUMENTO','DATA','DETALHAMENTO','COD_DOCUMENTO'],axis=1)
display(df)

,ANO,MES,SENADOR,TIPO_DESPESA,VALOR_REEMBOLSADO
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",6000.00
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,1500.00
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,6000.00
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,1000.00
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,2000.00
...,...,...,...,...,...
317958,2008,11,WELLINGTON SALGADO DE OLIVEIRA,"Locomoção, hospedagem, alimentação, combustíve...",6152.97
317959,2008,12,WELLINGTON SALGADO DE OLIVEIRA,"Aluguel de imóveis para escritório político, c...",3818.87
317960,2008,12,WELLINGTON SALGADO DE OLIVEIRA,Aquisição de material de consumo para uso no e...,886.66
317961,2008,12,WELLINGTON SALGADO DE OLIVEIRA,"Contratação de consultorias, assessorias, pesq...",7070.00


In [4]:
# Função para remover itens repetidos das listas criadas a partir das colunas do DataFrame, sem perder a ordem original
def removeRepetidos(lista):
    visto = set()
    adicionar_visto = visto.add
    return [x for x in lista if not (x in visto or adicionar_visto(x))]

In [5]:
# Criando listas a partir das colunas do DataFrame
lista_ano = df['ANO'].tolist()
lista_mes = df['MES'].tolist()
lista_senadores = df['SENADOR'].tolist()
lista_tipo_despesa = df['TIPO_DESPESA'].tolist()
lista_gasto_medio_mes = []
lista_gasto_medio_despesa = []

# Descobrindo os gastos médios por mês
for mes in removeRepetidos(lista_mes):
    df_mes = df[df['MES']==mes]
    gasto_medio_mes = (df_mes['VALOR_REEMBOLSADO'].sum())/len(df_mes)
    lista_gasto_medio_mes.append(gasto_medio_mes)

# Descobrindo os gastos médios por tipo de despesa
for tipo_despesa in removeRepetidos(lista_tipo_despesa):
    df_despesa = df_mes[df_mes['TIPO_DESPESA']==tipo_despesa]
    gasto_medio_despesa = (df_despesa['VALOR_REEMBOLSADO'].sum())/len(df_despesa)
    lista_gasto_medio_despesa.append(gasto_medio_despesa)

In [ ]:
# Gráfico de gastos médios por mês
fig = plt.figure(figsize=(15,8))
eixo = fig.add_axes([0, 0, 1, 1])
eixo.plot(removeRepetidos(lista_mes),lista_gasto_medio_mes, color="blue")
eixo.set_title(("Gasto médio por mês"), fontsize=25)
eixo.set_ylabel("Gasto em reais", fontsize=15)
eixo.set_xlabel("Mês", fontsize=15)
eixo.legend(["Gasto"], loc='lower right', fontsize=10)

In [ ]:
# Gráfico de gastos médios por tipo de despesa
fig = plt.figure(figsize=(15,8))
eixo = fig.add_axes([0, 0, 1, 1])
eixo.plot(lista_gasto_medio_despesa,removeRepetidos(lista_tipo_despesa), color='red')
eixo.set_title(("Gasto médio por tipo de despesa"), fontsize=25)
eixo.set_ylabel("Tipo de despesa", fontsize=20)
eixo.set_xlabel("Gasto em reais", fontsize=20)
eixo.legend(["Gasto"], loc='lower right', fontsize=15)

In [ ]:
# (tentativa 1) Descobrindo os gastos médios por ano e senador 
lista_desvio_padrao = []

for ano in removeRepetidos(lista_ano):
    df_ano = df[df['ANO']==ano]
    lista_senador_ano = df_ano['SENADOR'].tolist()
    lista_gasto_medio_senador_ano = []
    soma = 0
    variancia = 0
    for senador in removeRepetidos(lista_senador_ano):
        df_senador_ano = df_ano[df_ano['SENADOR']==senador]
        gasto_medio_senador_ano = (df_senador_ano['VALOR_REEMBOLSADO'].sum())/len(df_senador_ano)
        lista_gasto_medio_senador_ano.append(gasto_medio_senador_ano)
        soma += gasto_medio_senador_ano
    media = soma/len(df_ano)
    for x in lista_gasto_medio_senador_ano:
        variancia += ((x-media)**2)/(len(df_ano))
    desvio_padrao = variancia**(1/2)
    lista_desvio_padrao.append(desvio_padrao)

fig = plt.figure(figsize=(15,8))
eixo = fig.add_axes([0, 0, 1, 1])
eixo.plot(removeRepetidos(lista_ano), lista_desvio_padrao, color="green")
eixo.set_title("Desvio Padrão de cada ano", fontsize=25)
eixo.set_ylabel("Desvio Padrão", fontsize=15)
eixo.set_xlabel("Ano", fontsize=15)
eixo.legend(["Desvio Padrão"], loc="lower right", fontsize=10)

In [ ]:
# (tentativa 2) Descobrindo os gastos médios por senador e ano 
for senador in removeRepetidos(lista_senadores):
    df_senadores = df[df['SENADOR']==senador]
    lista_ano_senadores = df_senadores['ANO'].tolist()
    lista_desvio_padrao = []
    for ano in removeRepetidos(lista_ano_senadores):
        variancia = 0
        df_ano_senadores = df_senadores[df_senadores['ANO']==ano]
        lista_gasto_senadores = df_ano_senadores['VALOR_REEMBOLSADO'].tolist()
        gasto_medio_ano_senadores = df_ano_senadores['VALOR_REEMBOLSADO'].sum()/len(df_ano_senadores)
        for x in lista_gasto_senadores:
            variancia += ((x-gasto_medio_ano_senadores)**2)/(len(df_ano_senadores))
        desvio_padrao = variancia**(1/2)
        lista_desvio_padrao.append(desvio_padrao)

    fig = plt.figure(figsize=(15,8))
    eixo = fig.add_axes([0, 0, 1, 1])
    eixo.plot(removeRepetidos(lista_ano_senadores), lista_desvio_padrao, color="green")
    eixo.set_title(("Desvio Padrão do senador",senador,"em cada ano de mandato"), fontsize=25)
    eixo.set_ylabel("Desvio Padrão", fontsize=15)
    eixo.set_xlabel("Ano", fontsize=15)
    eixo.legend(["Desvio Padrão"], loc="lower right", fontsize=10)

In [ ]:
# (tentativa 3)
for ano in removeRepetidos(lista_ano):
    df_ano = df[df['ANO']==ano]
    lista_senadores_ano = df_ano['SENADOR'].tolist()
    for senador in removeRepetidos(lista_senadores_ano):
        lista_medias_despesas = []
        soma_medias = 0
        variancia = 0
        df_senadores_ano = df_ano[df_ano['SENADOR']==senador]
        lista_senadores_ano_despesa = df_senadores_ano['TIPO_DESPESA'].tolist()
        for tipo_despesa in removeRepetidos(lista_senadores_ano_despesa):
            df_senadores_ano_despesa = df_senadores_ano[df_senadores_ano['TIPO_DESPESA']==tipo_despesa]
            lista_gasto_senadores_ano_despesa = df_senadores_ano_despesa['VALOR_REEMBOLSADO'].tolist()
            gasto_medio_senadores_ano_despesa = df_senadores_ano_despesa['VALOR_REEMBOLSADO'].sum()/len(df_senadores_ano_despesa)
            lista_medias_despesas.append(gasto_medio_senadores_ano_despesa)
            soma_medias += gasto_medio_senadores_ano_despesa
        media_anual_senador = soma_medias/len(lista_medias_despesas)
        for x in lista_medias_despesas:
            variancia += ((x-media_anual_senador)**2)/len(lista_medias_despesas)
        desvio_padrao = variancia**(1/2)

        fig = plt.figure(figsize=(15,8))
        eixo = fig.add_axes([0, 0, 1, 1])
        eixo.plot(removeRepetidos(lista_senadores_ano_despesa), lista_medias_despesas, color="black")
        eixo.set_title((r"Média dos gastos do(a) senador(a)",senador,"por tipo de despesa no ano de",ano,"e com Desvio Padrão de",desvio_padrao), fontsize=25)
        eixo.set_ylabel("Média de gastos", fontsize=15)
        eixo.set_xlabel("Tipo de despesa", fontsize=15)
        eixo.legend(["Gasto"], loc="lower right", fontsize=10)


In [ ]:
#Deletável
df.TIPO_DESPESA.unique()
df.describe()

In [13]:
lista_gastos_senador = []

for senador in removeRepetidos(lista_senadores):
    df_senadores = df[df['SENADOR']==senador]
    gastos_senador = df_senadores['VALOR_REEMBOLSADO'].sum()
    lista_gastos_senador.append(gastos_senador)

df_senadores = pd.DataFrame(removeRepetidos(lista_senadores))
df_senadores.columns = ['SENADOR']
df_senadores['GASTOS_TOTAIS'] = lista_gastos_senador

display(df_senadores)
df_senadores.nlargest(10, 'GASTOS_TOTAIS')[['SENADOR','GASTOS_TOTAIS']]

,SENADOR,GASTOS_TOTAIS
0,ACIR GURGACZ,4522277.59
1,ALESSANDRO VIEIRA,1552531.31
2,ALEXANDRE SILVEIRA,243079.18
3,ALVARO DIAS,1110790.29
4,ANGELO CORONEL,1584308.03
...,...,...
286,FLÁVIO TORRES,46438.38
287,ROSEANA SARNEY,223301.59
288,ADA MELLO,59962.46
289,CARLOS DUNGA,59446.85


,SENADOR,GASTOS_TOTAIS
120,EDUARDO AMORIM,6228794.68
28,FERNANDO COLLOR,5222049.49
85,SÉRGIO PETECÃO,5085974.12
34,HUMBERTO COSTA,4663461.97
97,CIRO NOGUEIRA,4579717.29
75,RANDOLFE RODRIGUES,4571419.31
71,PAULO PAIM,4547595.40
0,ACIR GURGACZ,4522277.59
17,EDUARDO BRAGA,3981268.98
146,VANESSA GRAZZIOTIN,3870627.57
